In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/Aspect-based Sentiment Analysis/"

In [ ]:
import pickle
import pandas as pd
df = pd.read_csv(path+"review_train1.csv")
df_dev = pd.read_csv(path+'review_dev1.csv')
df_test = pd.read_csv(path+'review_test1.csv')

In [ ]:
data_train = df.review.to_list()
label_train = pickle.load(open(path+'VLSP2018/aspect_train.pkl','rb'))
data_dev = df_dev.review.tolist()
label_dev =  pickle.load(open(path+'VLSP2018/aspect_dev.pkl','rb'))
data_test = df_test.review.tolist()
label_test = pickle.load(open(path+'VLSP2018/aspect_test.pkl','rb'))

In [ ]:
!pip install -q keras-bert==0.85.0
!pip install -q keras-rectified-adam

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
pip install keras==2.3.1

In [ ]:
import codecs
import tensorflow as tf
import keras
import os
from keras_radam import RAdam
from keras import backend as K
from keras_bert import load_trained_model_from_checkpoint
import numpy as np

Using TensorFlow backend.


In [ ]:
!wget -q https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip -o multi_cased_L-12_H-768_A-12.zip

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  


In [ ]:
pretrained_path = 'multi_cased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [ ]:
SEQ_LEN = 256

In [ ]:
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    seq_len=SEQ_LEN,
    output_layer_num=4,
    trainable=True
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
import keras_bert
from keras.layers import Dense, Input, Flatten,SpatialDropout1D,Bidirectional
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda,MaxPool1D
from keras.models import Model
from keras.layers import Bidirectional,LSTM,GRU

In [ ]:
LR=3e-5

In [ ]:
class NonMasking(keras.layers.Layer):   
    def __init__(self, **kwargs):   
        self.supports_masking = True  
        super(NonMasking, self).__init__(**kwargs)   
  
    def build(self, input_shape):   
        input_shape = input_shape   
  
    def compute_mask(self, input, input_mask=None):   
        # do not pass the mask to the next layers   
        return None   
  
    def call(self, x, mask=None):   
        return x   
  
    def get_output_shape_for(self, input_shape):   
        return input_shape  

In [ ]:
def bert_rcnn():
  inputs = model.inputs
  bert_out = NonMasking()(model.outputs)

  bert_out = SpatialDropout1D(0.2)(bert_out)

  l_embedding = Lambda(lambda x: K.concatenate([K.zeros(shape=(K.shape(x)[0], 1, K.shape(x)[-1])),
                                                        x[:, :-1]], axis=1))(bert_out)
          
  r_embedding = Lambda(lambda x: K.concatenate([K.zeros(shape=(K.shape(x)[0], 1, K.shape(x)[-1])),
                                                        x[:, 1:]], axis=1))(bert_out)

  forward = LSTM(256, return_sequences=True)(l_embedding) 
  backward = LSTM(256, return_sequences=True, go_backwards=True)(r_embedding)
  backward = Lambda(lambda x: K.reverse(x, axes=1))(backward)

  together = [forward, bert_out , backward]

  together = Concatenate(axis=2)(together)

  semantic = Conv1D(256, kernel_size=1, activation="relu")(together)
  sentence_embed = Lambda(lambda x: K.max(x, axis=1))(semantic)

  dense_layer = Dense(768, activation='relu')(sentence_embed)
  dense_layer = Dense(512, activation='relu')(dense_layer)
  dense_layer = Dense(256, activation='relu')(dense_layer)
  preds = Dense(12, activation='sigmoid')(dense_layer)
  return Model(inputs, preds)

In [ ]:
def bert_base():
  inputs = model.inputs
  cls = keras_bert.layers.Extract(index=0)(model.output)
  dropout = Dropout(0.5)(cls)
  dense_layer = Dense(768, activation='relu')(dropout)
  dense_layer = Dense(512, activation='relu')(dense_layer)
  dense_layer = Dense(256, activation='relu')(dense_layer)
  preds = Dense(12, activation='sigmoid')(dense_layer)
  return Model(inputs, preds)

In [ ]:
def bert_textcnn():
  inputs = model.inputs
  bert_out = NonMasking()(model.outputs)

  bert_out = SpatialDropout1D(0.5)(bert_out)

  filter_lengths = [2, 3, 4, 5]
  conv_layers = []
  for filter_length in filter_lengths:
      conv_layer = Conv1D(filters=256, kernel_size=filter_length, padding='valid',
                          strides=1, activation='relu')(bert_out)
      maxpooling = MaxPool1D(pool_size=SEQ_LEN - filter_length + 1)(conv_layer)
      flatten = Flatten()(maxpooling)
      conv_layers.append(flatten)
  sentence_embed = Concatenate()(conv_layers)

  dense_layer = Dense(768, activation='relu')(sentence_embed)
  dense_layer = Dense(512, activation='relu')(dense_layer)
  dense_layer = Dense(256, activation='relu')(dense_layer)
  preds = Dense(12, activation='sigmoid')(dense_layer)
  return Model(inputs, preds)

In [ ]:
def bert_lstm():
  inputs = model.inputs
  lstm = Bidirectional(LSTM(units=256))(model.output)
  dense_layer = Dense(768, activation='relu')(lstm)
  dense_layer = Dense(512, activation='relu')(dense_layer)
  dense_layer = Dense(256, activation='relu')(dense_layer)
  preds = Dense(12, activation='sigmoid')(dense_layer)
  return Model(inputs, preds)

In [ ]:
token_dict = {}
with codecs.open(vocab_path, 'rb','utf-8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

In [ ]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict,cased=True)

In [ ]:
from sklearn.model_selection import train_test_split
def load_data(data, sentiments):
    global tokenizer
    indices = []
    for text in data:
      ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
      indices.append(ids)

    return [indices, np.zeros_like(indices)], np.array(sentiments)

In [ ]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
X_train,Y_train = load_data(data_train,label_train)
X_dev,Y_dev = load_data(data_dev,label_dev)
X_test,Y_test = load_data(data_test,label_test)

In [ ]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights
cw = calculating_class_weights(Y_train)

In [ ]:

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
aspect_model = bert_base()
#aspect_model = bert_rcnn()
#aspect_model = bert_lstm()
model.trainable = False
aspect_model.compile(loss='binary_crossentropy',
              optimizer=RAdam(learning_rate=LR),
              metrics=[f1])

In [ ]:
from keras.callbacks import ModelCheckpoint 
checkpoint = ModelCheckpoint(path+'checkpoint2', monitor='val_f1', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
aspect_model.fit(X_train,Y_train,epochs=1,batch_size=16,verbose = 1,validation_data=[X_test,Y_test],callbacks=callbacks_list, class_weight=cw)
model.trainable = True 
aspect_model.fit(X_train,Y_train,epochs=10,batch_size=16,verbose = 1,validation_data=[X_test,Y_test],callbacks=callbacks_list, class_weight=cw)       

Train on 2961 samples, validate on 500 samples
Epoch 1/1
2961/2961 [==============================] - 128s 43ms/step - loss: 0.5201 - f1: 0.4833 - val_loss: 0.5572 - val_f1: 0.6074

Epoch 00001: val_f1 improved from -inf to 0.60744, saving model to /content/drive/My Drive/Aspect-based Sentiment Analysis/checkpoint2
Train on 2961 samples, validate on 500 samples
Epoch 1/10
2961/2961 [==============================] - 114s 38ms/step - loss: 0.3817 - f1: 0.6294 - val_loss: 0.5363 - val_f1: 0.6557

Epoch 00001: val_f1 improved from 0.60744 to 0.65572, saving model to /content/drive/My Drive/Aspect-based Sentiment Analysis/checkpoint2
Epoch 2/10
2961/2961 [==============================] - 114s 38ms/step - loss: 0.3474 - f1: 0.6848 - val_loss: 0.4984 - val_f1: 0.6878

Epoch 00002: val_f1 improved from 0.65572 to 0.68784, saving model to /content/drive/My Drive/Aspect-based Sentiment Analysis/checkpoint2
Epoch 3/10
2961/2961 [==============================] - 114s 38ms/step - loss: 0.3111 - 

In [ ]:
aspect_model.fit(X_train,Y_train,epochs=10,batch_size=16,verbose = 1,validation_data=[X_test,Y_test],callbacks=callbacks_list, class_weight=cw)       

Train on 2961 samples, validate on 500 samples
Epoch 1/10
2961/2961 [==============================] - 114s 38ms/step - loss: 0.0996 - f1: 0.9248 - val_loss: 0.5784 - val_f1: 0.7499

Epoch 00001: val_f1 did not improve from 0.77044
Epoch 2/10
2961/2961 [==============================] - 113s 38ms/step - loss: 0.0835 - f1: 0.9377 - val_loss: 0.6163 - val_f1: 0.7513

Epoch 00002: val_f1 did not improve from 0.77044
Epoch 3/10
2961/2961 [==============================] - 113s 38ms/step - loss: 0.0663 - f1: 0.9510 - val_loss: 0.6521 - val_f1: 0.7605

Epoch 00003: val_f1 did not improve from 0.77044
Epoch 4/10
2961/2961 [==============================] - 114s 38ms/step - loss: 0.0640 - f1: 0.9524 - val_loss: 0.6495 - val_f1: 0.7697

Epoch 00004: val_f1 did not improve from 0.77044
Epoch 5/10
2961/2961 [==============================] - 114s 38ms/step - loss: 0.0463 - f1: 0.9673 - val_loss: 0.6634 - val_f1: 0.7678

Epoch 00005: val_f1 did not improve from 0.77044
Epoch 6/10
2961/2961 [======

In [ ]:
aspect_model.load_weights(path+'checkpoint2')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
# bert-rcnn
predictions= aspect_model.predict(X_test)
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(Y_test, pred, average='micro')
    recall = recall_score(Y_test, pred, average='micro')
    f1 = f1_score(Y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Micro-average quality numbers
Precision: 0.7209, Recall: 0.8446, F1-measure: 0.7778
Micro-average quality numbers
Precision: 0.7496, Recall: 0.8206, F1-measure: 0.7835
Micro-average quality numbers
Precision: 0.7663, Recall: 0.8065, F1-measure: 0.7859
Micro-average quality numbers
Precision: 0.7784, Recall: 0.7958, F1-measure: 0.7870
Micro-average quality numbers
Precision: 0.7860, Recall: 0.7821, F1-measure: 0.7841
Micro-average quality numbers
Precision: 0.7989, Recall: 0.7751, F1-measure: 0.7868
Micro-average quality numbers
Precision: 0.8085, Recall: 0.7594, F1-measure: 0.7832
Micro-average quality numbers
Precision: 0.8210, Recall: 0.7453, F1-measure: 0.7814
Micro-average quality numbers
Precision: 0.8349, Recall: 0.7131, F1-measure: 0.7692


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
# bert-base
predictions= aspect_model.predict(X_test)
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(Y_test, pred, average='micro')
    recall = recall_score(Y_test, pred, average='micro')
    f1 = f1_score(Y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Micro-average quality numbers
Precision: 0.7204, Recall: 0.8487, F1-measure: 0.7793
Micro-average quality numbers
Precision: 0.7522, Recall: 0.8268, F1-measure: 0.7877
Micro-average quality numbers
Precision: 0.7656, Recall: 0.8127, F1-measure: 0.7884
Micro-average quality numbers
Precision: 0.7755, Recall: 0.8024, F1-measure: 0.7887
Micro-average quality numbers
Precision: 0.7856, Recall: 0.7908, F1-measure: 0.7882
Micro-average quality numbers
Precision: 0.7976, Recall: 0.7788, F1-measure: 0.7881
Micro-average quality numbers
Precision: 0.8066, Recall: 0.7602, F1-measure: 0.7827
Micro-average quality numbers
Precision: 0.8159, Recall: 0.7383, F1-measure: 0.7752
Micro-average quality numbers
Precision: 0.8303, Recall: 0.7040, F1-measure: 0.7620


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import itertools
matplotlib.rcParams.update({'font.size': 16})
labels = [0,1]
cn = confusion_matrix(Y_test,y_pred,labels=labels)

NameError: ignored

In [ ]:
#@title plot confusion matrix
def plot_confusion_matrix(cm,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True,
                          target_names=None,
                          path_file='1.svg'):
    
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(path_file,format='svg')
    plt.show()

In [ ]:
plot_confusion_matrix(cn,title='Data vreview',normalize=False,target_names=labels,path_file=path+'bert_rcnn_vreview_cm.svg')